<a href="https://colab.research.google.com/github/gitswathig/assignment3/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **task 2**

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded.keys())))
df.head()


Saving Dataset .csv to Dataset .csv


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [7]:
# Fill cuisines
df['Cuisines'] = df.get('Cuisines', '').fillna('')

# Possible cost columns in different datasets
possible_cost_cols = [
    "Cost", "Average Cost for two", "Price range",
    "approx_cost(for two people)", "Cost for two",
    "Cost(for two people)", "Average cost for two"
]

cost_col = None
for col in possible_cost_cols:
    if col in df.columns:
        cost_col = col
        break

if cost_col is None:

    numeric_candidates = [c for c in df.columns if df[c].dtype in ['int64','float64']]

    for c in numeric_candidates:
        if 'cost' in c.lower() or 'price' in c.lower():
            cost_col = c
            break

if cost_col is None:
    raise ValueError("No cost column found. Possible names tried: " + ", ".join(possible_cost_cols))

print("Detected cost column:", cost_col)

# Detect rating column
possible_rating_cols = ["Aggregate rating", "Rating", "rate", "aggregate_rating"]
rating_col = None
for col in possible_rating_cols:
    if col in df.columns:
        rating_col = col
        break
if rating_col is None:

    for c in df.columns:
        if 'rating' in c.lower():
            rating_col = c
            break
if rating_col is None:
    # default fallback: create rating as zeros
    rating_col = None
    print("Warning: No rating column found. 'Rating' will be set to 0s.")
else:
    print("Detected rating column:", rating_col)


df['Cost'] = pd.to_numeric(df[cost_col], errors='coerce').fillna(df[cost_col].median())
if rating_col is not None:
    df['Rating'] = pd.to_numeric(df[rating_col], errors='coerce').fillna(df[rating_col].median())
else:
    df['Rating'] = 0.0

# duplicates and basic sanity check
df.drop_duplicates(inplace=True)
print("Rows:", len(df), "| Columns now include unified 'Cost' and 'Rating'")


Detected cost column: Average Cost for two
Detected rating column: Aggregate rating
Rows: 9551 | Columns now include unified 'Cost' and 'Rating'


In [9]:

import re
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

def clean_text(s):
    if pd.isnull(s): return ""
    s = str(s).lower()
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    tokens = [t for t in s.split() if t not in STOPWORDS and len(t) > 1]
    return " ".join(tokens)

df['text_features'] = (df.get('Cuisines','').astype(str) + " " + df.get('Restaurant Name','').astype(str)).apply(clean_text)
df['text_features'].head()


,text_features
0,french japanese desserts le petit souffle
1,japanese izakaya kikufuji
2,seafood asian filipino indian heat edsa shangr...
3,japanese sushi ooma
4,japanese korean sambo kojin


In [10]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(df['text_features'])   # sparse matrix

# Scale Cost to [0,1]
scaler = MinMaxScaler()
cost_scaled = scaler.fit_transform(df[['Cost']])  # returns np.ndarray shape (n,1)

# Combine TF-IDF (dense) + cost_scaled

final_features = np.hstack([tfidf_matrix.toarray(), cost_scaled])
print("final_features shape:", final_features.shape)


final_features shape: (9551, 2001)


In [11]:

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(final_features)
print("similarity_matrix shape:", similarity_matrix.shape)


similarity_matrix shape: (9551, 9551)


In [12]:

def recommend_restaurant(name, top_n=10):
    name = name.lower().strip()

    # Column may be 'Restaurant Name' or variations
    name_col = None
    for col in df.columns:
        if "name" in col.lower():
            name_col = col
            break

    if name_col is None:
        raise ValueError("No restaurant name column found.")

    # Find matching rows
    matches = df[df[name_col].str.lower() == name]

    if matches.empty:
        return f"No restaurant named '{name}' found."

    idx = matches.index[0]   # take first match

    # Similarity scores
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Top N
    top_indices = [i for i, score in scores[1:top_n+1]]

    result = df.iloc[top_indices][[name_col, 'Cuisines', 'Cost', 'Rating']]
    return result.reset_index(drop=True)

print("Recommendation function ready!")


Recommendation function ready!


In [14]:

def recommend_by_preferences(preferred_cuisine=None, max_cost=None, min_rating=None, top_n=10):
    df_copy = df.copy()

    # Normalize text columns
    if 'Cuisines' in df_copy.columns:
        df_copy['Cuisines'] = df_copy['Cuisines'].astype(str).str.lower()

    # Apply filters
    if preferred_cuisine:
        preferred_cuisine = preferred_cuisine.lower().strip()
        df_copy = df_copy[df_copy['Cuisines'].str.contains(preferred_cuisine, na=False)]

    if max_cost:
        df_copy = df_copy[df_copy['Cost'] <= max_cost]

    if min_rating:
        df_copy = df_copy[df_copy['Rating'] >= min_rating]

    # If no match found
    if df_copy.empty:
        return f"No restaurants found matching the given preferences."

    # Sort by rating
    df_copy = df_copy.sort_values(by='Rating', ascending=False)

    # Select important columns
    cols = [col for col in df_copy.columns if "name" in col.lower()]
    name_col = cols[0]

    return df_copy[[name_col, 'Cuisines', 'Cost', 'Rating']].head(top_n).reset_index(drop=True)


print("Preference-based recommender ready!")


Preference-based recommender ready!


In [15]:


recommend_by_preferences(
    preferred_cuisine="Italian",
    max_cost=500,
    min_rating=4.0,
    top_n=5
)


,Restaurant Name,Cuisines,Cost,Rating
0,Mazzaro's Italian Market,"italian, deli",10,4.9
1,Ella's Americana Folk Art Cafe,"international, italian, southern",40,4.8
2,Gemelli Cucina Bar,"contemporary, italian",450,4.8
3,New Yorker Deli & Pizzeria,"italian, pizza, sandwich",10,4.6
4,Baduzzi,italian,120,4.6
